In [1]:
from pybaseball import statcast
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import warnings
warnings.filterwarnings("ignore", message="X does not have valid feature names")

In [3]:
#savant_df2016 = statcast(start_dt = "2016-03-28", end_dt = "2016-11-15")

In [4]:
#savant_df2017 = statcast(start_dt = "2017-03-28", end_dt = "2017-11-15")

In [5]:
#savant_df2018 = statcast(start_dt = "2018-03-28", end_dt = "2018-11-15")

In [6]:
#savant_df2021 = statcast(start_dt = "2021-03-28", end_dt = "2021-11-15")

In [7]:
#savant_df2022 = statcast(start_dt = "2022-03-28", end_dt = "2022-11-15")

In [8]:
#expected = pd.concat([savant_df2016, savant_df2017, savant_df2018, savant_df2021, savant_df2022], axis = 0)

In [9]:
expected = pd.read_csv("Expected.csv")

/var/folders/bs/6lp27nzn3wvg1ygrkwrcl96w0000gn/T/ipykernel_72683/1292578522.py:1: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  expected = pd.read_csv("Expected.csv")


In [10]:
# Converts the Induced Vertical Break and Horizontal Break to inches from feet
expected['IVB'] = expected['pfx_z'] * 12
expected['HB'] = expected['pfx_x'] * 12
expected['ABS_HB'] = np.abs(expected["HB"])
expected['ABS_RelSide'] = np.abs(expected["release_pos_x"])

expected = expected.rename(columns={'player_name': 'Name', "pitch_type" : "Pitch Type", "release_pos_z" : "RelHeight"})
expected

,Pitch Type,game_date,release_speed,release_pos_x,RelHeight,Name,batter,pitcher,events,description,...,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,IVB,HB,ABS_HB,ABS_RelSide
0,CU,2016-11-02,76.6,2.93,6.59,Mike Montgomery,492841,543557,field_out,hit_into_play,...,8.0,Standard,Standard,317.0,-0.096,-0.180,-15.84,-9.36,9.36,2.93
1,CU,2016-11-02,74.3,3.04,6.51,Mike Montgomery,492841,543557,NaN,called_strike,...,8.0,Standard,Standard,318.0,0.000,-0.040,-15.48,-8.16,8.16,3.04
2,FF,2016-11-02,94.9,-1.29,6.46,Carl Edwards Jr.,434658,605218,single,hit_into_play,...,8.0,Standard,Standard,184.0,0.050,0.669,17.52,0.48,0.48,1.29
3,FF,2016-11-02,95.6,-1.28,6.51,Carl Edwards Jr.,434658,605218,NaN,ball,...,8.0,Standard,Standard,191.0,0.002,0.031,17.64,-2.16,2.16,1.28
4,FF,2016-11-02,94.8,-1.13,6.30,Carl Edwards Jr.,446386,605218,walk,ball,...,8.0,Standard,Standard,185.0,0.030,0.067,14.52,2.76,2.76,1.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3658661,NaN,2022-03-28,NaN,NaN,NaN,Patrick Corbin,621563,571578,NaN,ball,...,0.0,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN
3658662,NaN,2022-03-28,NaN,NaN,NaN,Patrick Corbin,621563,571578,NaN,ball,...,0.0,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN
3658663,NaN,2022-03-28,NaN,NaN,NaN,Patrick Corbin,624585,571578,strikeout,swinging_strike,...,0.0,NaN,NaN,NaN,0.022,NaN,NaN,NaN,NaN,NaN
3658664,NaN,2022-03-28,NaN,NaN,NaN,Patrick Corbin,624585,571578,NaN,swinging_strike,...,0.0,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN


In [11]:
df_swing = expected[expected['description'].isin(['swinging_strike', 'swinging_strike_blocked', 'hit_into_play', 'foul'])]
df_swing

,Pitch Type,game_date,release_speed,release_pos_x,RelHeight,Name,batter,pitcher,events,description,...,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,IVB,HB,ABS_HB,ABS_RelSide
0,CU,2016-11-02,76.6,2.93,6.59,Mike Montgomery,492841,543557,field_out,hit_into_play,...,8.0,Standard,Standard,317.0,-0.096,-0.180,-15.84,-9.36,9.36,2.93
2,FF,2016-11-02,94.9,-1.29,6.46,Carl Edwards Jr.,434658,605218,single,hit_into_play,...,8.0,Standard,Standard,184.0,0.050,0.669,17.52,0.48,0.48,1.29
9,CU,2016-11-02,81.3,-0.98,6.39,Carl Edwards Jr.,608070,605218,field_out,hit_into_play,...,8.0,Standard,Standard,44.0,-0.029,-0.144,-11.88,13.92,13.92,0.98
12,CU,2016-11-02,82.4,-1.17,6.30,Carl Edwards Jr.,608070,605218,NaN,foul,...,8.0,Standard,Standard,51.0,0.000,-0.037,-12.72,12.96,12.96,1.17
14,FF,2016-11-02,95.2,-1.16,6.37,Carl Edwards Jr.,435063,605218,strikeout,swinging_strike,...,8.0,Standard,Standard,165.0,-0.044,-0.208,14.04,5.28,5.28,1.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3658657,NaN,2022-03-28,NaN,NaN,NaN,Edward Cabrera,514917,665795,field_out,hit_into_play,...,0.0,NaN,NaN,NaN,-0.021,NaN,NaN,NaN,NaN,NaN
3658658,NaN,2022-03-28,NaN,NaN,NaN,Patrick Corbin,643265,571578,grounded_into_double_play,hit_into_play,...,0.0,NaN,NaN,NaN,0.050,NaN,NaN,NaN,NaN,NaN
3658663,NaN,2022-03-28,NaN,NaN,NaN,Patrick Corbin,624585,571578,strikeout,swinging_strike,...,0.0,NaN,NaN,NaN,0.022,NaN,NaN,NaN,NaN,NaN
3658664,NaN,2022-03-28,NaN,NaN,NaN,Patrick Corbin,624585,571578,NaN,swinging_strike,...,0.0,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN


In [12]:
df_swing['Whiff'] = df_swing['description'].apply(lambda x: 1 if x == 'swinging_strike' else 0)
df_swing

/var/folders/bs/6lp27nzn3wvg1ygrkwrcl96w0000gn/T/ipykernel_72683/3217903474.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_swing['Whiff'] = df_swing['description'].apply(lambda x: 1 if x == 'swinging_strike' else 0)


,Pitch Type,game_date,release_speed,release_pos_x,RelHeight,Name,batter,pitcher,events,description,...,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,IVB,HB,ABS_HB,ABS_RelSide,Whiff
0,CU,2016-11-02,76.6,2.93,6.59,Mike Montgomery,492841,543557,field_out,hit_into_play,...,Standard,Standard,317.0,-0.096,-0.180,-15.84,-9.36,9.36,2.93,0
2,FF,2016-11-02,94.9,-1.29,6.46,Carl Edwards Jr.,434658,605218,single,hit_into_play,...,Standard,Standard,184.0,0.050,0.669,17.52,0.48,0.48,1.29,0
9,CU,2016-11-02,81.3,-0.98,6.39,Carl Edwards Jr.,608070,605218,field_out,hit_into_play,...,Standard,Standard,44.0,-0.029,-0.144,-11.88,13.92,13.92,0.98,0
12,CU,2016-11-02,82.4,-1.17,6.30,Carl Edwards Jr.,608070,605218,NaN,foul,...,Standard,Standard,51.0,0.000,-0.037,-12.72,12.96,12.96,1.17,0
14,FF,2016-11-02,95.2,-1.16,6.37,Carl Edwards Jr.,435063,605218,strikeout,swinging_strike,...,Standard,Standard,165.0,-0.044,-0.208,14.04,5.28,5.28,1.16,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3658657,NaN,2022-03-28,NaN,NaN,NaN,Edward Cabrera,514917,665795,field_out,hit_into_play,...,NaN,NaN,NaN,-0.021,NaN,NaN,NaN,NaN,NaN,0
3658658,NaN,2022-03-28,NaN,NaN,NaN,Patrick Corbin,643265,571578,grounded_into_double_play,hit_into_play,...,NaN,NaN,NaN,0.050,NaN,NaN,NaN,NaN,NaN,0
3658663,NaN,2022-03-28,NaN,NaN,NaN,Patrick Corbin,624585,571578,strikeout,swinging_strike,...,NaN,NaN,NaN,0.022,NaN,NaN,NaN,NaN,NaN,1
3658664,NaN,2022-03-28,NaN,NaN,NaN,Patrick Corbin,624585,571578,NaN,swinging_strike,...,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,1


In [13]:
df_swing = df_swing[["Name", "Pitch Type", "release_speed", "release_spin_rate", "IVB", "ABS_HB", "RelHeight", "ABS_RelSide", "release_extension", "Whiff"]].dropna()
df_swing

,Name,Pitch Type,release_speed,release_spin_rate,IVB,ABS_HB,RelHeight,ABS_RelSide,release_extension,Whiff
0,Mike Montgomery,CU,76.6,2324.0,-15.84,9.36,6.59,2.93,6.2,0
2,Carl Edwards Jr.,FF,94.9,2636.0,17.52,0.48,6.46,1.29,6.8,0
9,Carl Edwards Jr.,CU,81.3,2468.0,-11.88,13.92,6.39,0.98,6.4,0
12,Carl Edwards Jr.,CU,82.4,2695.0,-12.72,12.96,6.30,1.17,5.4,0
14,Carl Edwards Jr.,FF,95.2,2703.0,14.04,5.28,6.37,1.16,6.9,1
...,...,...,...,...,...,...,...,...,...,...
3658515,Adam Wainwright,FC,84.0,2349.0,8.88,6.24,6.32,1.26,6.4,0
3658518,Adam Wainwright,SI,88.4,2063.0,12.84,14.04,6.21,1.13,6.5,0
3658519,Adam Wainwright,CH,81.9,1610.0,5.04,10.56,6.26,1.34,6.2,1
3658520,Adam Wainwright,SI,87.7,2088.0,9.60,11.52,6.26,1.29,6.4,0


In [14]:
df_sinker = df_swing[(df_swing["Pitch Type"] == "SI")]
df_cutter = df_swing[df_swing["Pitch Type"] == "FC"]
df_fastball = df_swing[df_swing["Pitch Type"] == "FF"]
df_changeup = df_swing[df_swing["Pitch Type"] == "CH"]
df_splitter = df_swing[(df_swing["Pitch Type"] == "FS") | (df_swing["Pitch Type"] == "FO")]
df_curveball = df_swing[(df_swing["Pitch Type"] == "CU") | (df_swing["Pitch Type"] == "KC")]
df_slider = df_swing[(df_swing["Pitch Type"] == "SL") | (df_swing["Pitch Type"] == "SV")]
df_sweeper = df_swing[df_swing["Pitch Type"] == "ST"]

## Sinker

In [15]:
SIfeature = df_sinker.drop(["Name", "Pitch Type", "Whiff"], axis=1)
SIlabel = df_sinker['Whiff']

SIrf = RandomForestRegressor(n_estimators = 15, random_state = 42).fit(SIfeature, SIlabel)

## Cutter

In [16]:
CTfeature = df_cutter.drop(["Name", "Pitch Type", "Whiff"], axis=1)
CTlabel = df_cutter['Whiff']

CTrf = RandomForestRegressor(n_estimators = 15, random_state = 42).fit(CTfeature, CTlabel)

## 4-Seam

In [17]:
FBfeature = df_fastball.drop(["Name", "Pitch Type", "Whiff"], axis=1)
FBlabel = df_fastball['Whiff']

FBrf = RandomForestRegressor(n_estimators = 30, random_state = 42).fit(FBfeature, FBlabel)

## Changeup

In [18]:
CHfeature = df_changeup.drop(["Name", "Pitch Type", "Whiff"], axis=1)
CHlabel = df_changeup['Whiff']

CHrf = RandomForestRegressor(n_estimators = 15, random_state = 42).fit(CHfeature, CHlabel)

## Splitter

In [19]:
SPfeature = df_splitter.drop(["Name", "Pitch Type", "Whiff"], axis=1)
SPlabel = df_splitter['Whiff']

SPrf = RandomForestRegressor(n_estimators = 10, random_state = 42).fit(SPfeature, SPlabel)

## Curveball

In [20]:
CUfeature = df_curveball.drop(["Name", "Pitch Type", "Whiff"], axis=1)
CUlabel = df_curveball['Whiff']

CUrf = RandomForestRegressor(n_estimators = 20, random_state = 42).fit(CUfeature, CUlabel)

## Slider

In [21]:
SLfeature = df_slider.drop(["Name", "Pitch Type", "Whiff"], axis=1)
SLlabel = df_slider['Whiff']

SLrf = RandomForestRegressor(n_estimators = 25, random_state = 42).fit(SLfeature, SLlabel)

## Sweeper

In [22]:
STfeature = df_sweeper.drop(["Name", "Pitch Type", "Whiff"], axis=1)
STlabel = df_sweeper['Whiff']

STrf = RandomForestRegressor(n_estimators = 10, random_state = 42).fit(STfeature, STlabel)

## 2023 DF

In [23]:
savant_df = statcast(start_dt = "2023-03-30", end_dt = "2023-11-15")

savant_df['IVB'] = savant_df['pfx_z'] * 12
savant_df['HB'] = savant_df['pfx_x'] * 12
savant_df['ABS_HB'] = np.abs(savant_df["HB"])
savant_df['ABS_RelSide'] = np.abs(savant_df["release_pos_x"])

savant_df = savant_df.rename(columns={'player_name': 'Name', "pitch_type" : "Pitch Type", "release_pos_z" : "RelHeight"})

def flip_names(name):
    first_name, last_name = name.split(", ")
    return f"{last_name} {first_name}"

savant_df['Name'] = savant_df["Name"].apply(flip_names)

savant_df = savant_df[["Name", "Pitch Type", "release_speed", "release_spin_rate", "IVB", "ABS_HB", "RelHeight", "ABS_RelSide", "release_extension"]].dropna()
savant_df = savant_df.reset_index(drop = True).dropna()

savant_df

This is a large query, it may take a moment to complete


100%|██████████| 231/231 [02:05<00:00,  1.84it/s]


,Name,Pitch Type,release_speed,release_spin_rate,IVB,ABS_HB,RelHeight,ABS_RelSide,release_extension
0,Jhoan Duran,FF,101.3,2169,12.0,12.6,5.92,2.13,6.2
1,Jhoan Duran,FF,101.7,2217,12.48,13.32,5.92,2.05,6.2
2,Jhoan Duran,FF,100.2,2144,13.32,12.24,5.83,2.14,6.3
3,Jhoan Duran,CU,87.0,2485,-8.16,3.84,5.91,1.94,6.4
4,Jhoan Duran,CU,86.4,2593,-8.76,4.08,5.96,2.03,6.2
...,...,...,...,...,...,...,...,...,...
715264,Luis Castillo,SI,95.3,2120,5.52,20.28,5.24,3.11,5.6
715265,Luis Castillo,FF,94.2,2284,13.92,15.36,5.29,3.16,5.6
715266,Luis Castillo,FF,94.8,2122,14.28,9.6,5.3,3.12,5.4
715267,Luis Castillo,SI,95.1,2126,4.92,20.52,5.22,3.2,5.5


In [24]:
player = savant_df[savant_df["Name"] == "Corbin Burnes"].reset_index(drop = True).dropna()
player

,Name,Pitch Type,release_speed,release_spin_rate,IVB,ABS_HB,RelHeight,ABS_RelSide,release_extension
0,Corbin Burnes,FC,94.2,2724,9.12,4.44,5.6,0.89,6.7
1,Corbin Burnes,FC,94.1,2818,6.0,6.72,5.7,0.83,6.6
2,Corbin Burnes,CU,80.6,2866,-13.92,11.76,5.8,0.44,6.7
3,Corbin Burnes,FC,94.8,2833,6.6,8.28,5.7,0.87,6.5
4,Corbin Burnes,FC,94.2,2712,6.84,6.48,5.63,1.03,6.6
...,...,...,...,...,...,...,...,...,...
3143,Corbin Burnes,SL,87.9,2991,0.84,6.12,5.87,0.93,6.5
3144,Corbin Burnes,FC,96.3,2717,17.16,3.96,6.05,0.79,6.3
3145,Corbin Burnes,SL,89.4,2865,3.0,5.76,5.95,0.79,6.4
3146,Corbin Burnes,FC,95.2,2617,12.12,1.68,5.9,0.87,6.3


In [25]:
stuff = player.drop(["Name", "Pitch Type"], axis = 1)

In [26]:
for i in range(len(player)):
    pitch_type = player.loc[i, 'Pitch Type']
    
    if pitch_type == "SI":
        player.at[i, "Stuff"] = SIrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_sinker["Whiff"] == 1) / len(df_sinker["Whiff"])) * 100
  
    elif pitch_type == "FC":
        player.at[i, "Stuff"] = CTrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_cutter["Whiff"] == 1) / len(df_cutter["Whiff"])) * 100

    elif pitch_type == "FF":
        player.at[i, "Stuff"] = FBrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_fastball["Whiff"] == 1) / len(df_fastball["Whiff"])) * 100

    elif pitch_type == "CH":
        player.at[i, "Stuff"] = CHrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_changeup["Whiff"] == 1) / len(df_changeup["Whiff"])) * 100

    elif pitch_type == "SP" or pitch_type == "FO":
        player.at[i, "Stuff"] = SPrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_splitter["Whiff"] == 1) / len(df_splitter["Whiff"])) * 100

    elif pitch_type == "CU" or pitch_type == "KC":
        player.at[i, "Stuff"] = CUrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_curveball["Whiff"] == 1) / len(df_curveball["Whiff"])) * 100

    elif pitch_type == "SL" or pitch_type == "SV":
        player.at[i, "Stuff"] = SLrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_slider["Whiff"] == 1) / len(df_slider["Whiff"])) * 100

    elif pitch_type == "ST":
        player.at[i, "Stuff"] = STrf.predict(stuff.iloc[i].values.reshape(1, -1))[0] / (np.sum(df_sweeper["Whiff"] == 1) / len(df_sweeper["Whiff"])) * 100

In [27]:
player_grouped = player.groupby(['Name', 'Pitch Type']).agg(
    Pitches = ('Pitch Type', 'count'), 
    Velo = ('release_speed', 'mean'),
    Max_Velo = ('release_speed', 'max'),
    SpinRate = ('release_spin_rate', 'mean'),
    IVB = ('IVB', 'mean'),
    ABS_HB = ('ABS_HB', 'mean'),
    Release_Height = ('RelHeight', 'mean'),
    Extension = ('release_extension', 'mean'),
    Stuff = ('Stuff', 'mean')).round(2)

player_grouped

Pitches   Velo  Max_Velo  SpinRate    IVB  ABS_HB  \
Name          Pitch Type                                                      
Corbin Burnes CH              351   89.3      91.9   2049.36    7.0   13.78   
              CU              542  79.88      83.7   2795.47 -11.02   10.38   
              FC             1746  94.44      97.3    2656.5  11.16    3.57   
              SI              241  95.32      97.8   2492.25   10.9   10.62   
              SL              268  85.95      91.8   2879.55   1.55   12.79   

                          Release_Height  Extension   Stuff  
Name          Pitch Type                                     
Corbin Burnes CH                    5.69       6.62  126.69  
              CU                    6.06       6.53  131.88  
              FC                     5.9       6.57  121.30  
              SI                    5.88       6.63  157.67  
              SL                    5.78       6.56  138.57